In [4]:
import numpy as np
import pandas as pd
from zipfile import ZipFile

In [104]:
# specifying the zip file name
file_name = "/mnt/d/Mexico_Births/sinac_2020.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    name = zip.filelist
    data = pd.read_csv(zip.open(name[0].filename) )

/home/argarcia/anaconda3/envs/test/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (49,59) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [91]:
data = data[["EDAD","ESTADOCONYUGAL","ESCOLARIDAD","CLAVEOCUPACIONHABITUAL",]]

In [80]:
@wrapper
def test(text):
    print("this is a test "+text)

In [78]:
test("/mnt/d/Mexico_Births/sinac_2020.zip")

this is a test /mnt/d/Mexico_Births/sinac_2020.zip
